In [1]:
import json

from redatam import bajar_query, soup_to_dict, redatam_dict_to_geojson
from keplergl import KeplerGl 
from pprint import pprint

In [2]:

# Consulta para NBI Argentina
# Cantidad de casas que no posee inodoro o letrina HOGAR.H1510 = 2
query_nbi_ii_arg = 'HOGAR.H1510 = 2'


# Consulta para NBI Uruguay

# NBI II criterio agua potable:
#    Un hogar particular tendrá una necesidad básica insatisfecha con relación al abastecimiento de agua potable si esta
#    no llega por cañería dentro de la vivienda o si, en caso de que lo haga, el origen no sea red general ni pozo 
#    surgente protegido.
query_agua_potable_uy = 'HOGAR.H1409 > 3 AND HOGAR.H1308 <> 1'


# NBI II criterio excretas:
#    Un hogar particular tendrá necesidades básicas insatisfechas en saneamiento si cumple al menos una de las
#    siguientes condiciones: 
#        a) no accede a baño;
#        b) accediendo, su uso no es exclusivo del hogar o;
#        c) la evacuación del servicio sanitario no es a red general, fosa séptica o pozo negro.

query_excretas_uy_a = query_nbi_ii_arg
query_excretas_uy_b = 'HOGAR.H1712 = 4 OR HOGAR.H1813 = 2'

query_nbi_ii_uy = "({0}) OR ({1}) OR ({2})".format(query_agua_potable_uy, query_excretas_uy_a, query_excretas_uy_b)

In [3]:
with open('radios_censales_santa_fe.json') as jsonfile:
    radios_santa_fe = json.load(jsonfile)

In [4]:
nbi_ii_arg = soup_to_dict(bajar_query(query_nbi_ii_arg))

In [5]:
nbi_ii_uy = soup_to_dict(bajar_query(query_nbi_ii_uy))

In [6]:
total_hogares = soup_to_dict(bajar_query(''))

In [7]:
nbi_ii_relativo = {}
for radio, valores in total_hogares.items():
    hogares = valores['total']
    if not hogares:
        nbi_ii_relativo[radio] = {'nbi_ii_arg': 0.0, 'nbi_ii_uy': 0.0}        
        print("Radio {} vacío".format(radio))
        continue
    try:
        nbi_arg = nbi_ii_arg[radio]['total']
    except KeyError:
        print('Sin datos para radio {} en metodología Argentina'.format(radio))
        nbi_arg = 0
    try:
        nbi_uy = nbi_ii_uy[radio]['total']
    except KeyError:
        print('Sin datos para radio {} en metodología Uruguay'.format(radio))
        nbi_uy = 0
    nbi_ii_relativo[radio] = {
        'nbi_ii_arg': round(nbi_arg/hogares, 4)*100,
        'nbi_ii_uy': round(nbi_uy/hogares, 4)*100
    }

Radio 820490811 vacío
Radio 820630321 vacío
Radio 820630410 vacío
Radio 820632004 vacío
Radio 820633405 vacío
Radio 820841019 vacío
Radio 820844105 vacío
Radio 821330110 vacío
Radio 821330302 vacío
Radio 821330519 vacío
Radio 821330521 vacío


In [8]:
geojson_nbi_ii = redatam_dict_to_geojson(nbi_ii_relativo, radios_santa_fe)

Radio censal no encontrado en REDATAM: 821330631
Radio censal no encontrado en REDATAM: 820070101


In [9]:
with open('kepler.config.json') as jsonfile:
    kepler_config = json.load(jsonfile)

In [10]:
mapa = KeplerGl(height=700, config=kepler_config)
mapa.add_data(geojson_nbi_ii, name="NBI II")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [11]:
mapa

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '67yqg3p', 'type': …

In [23]:
with open('kepler.config.json', 'w') as jsonfile:
    json.dump(mapa.config, jsonfile)